In [1]:
# Imports for making dataframe
import pandas as pd
import numpy as np

In [2]:
# Imports for crawling for HTML -> JSON
import requests
import json

In [3]:
# Import for finding texts in HTML JSON
from bs4 import BeautifulSoup

In [4]:
# Imports for seeing full text reviews
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [5]:
# Load Amazon data to find reviews of same books from Goodreads
amazon = pd.read_csv('amazonData.csv')
amazon

,overall,reviewTime,asin,reviewText
0,5.0,"03 30, 2005",0001713353,"The King, the Mice and the Cheese by Nancy Gur..."
1,5.0,"12 27, 2012",0001713353,This was one of my favorites when I was a smal...
2,5.0,"08 15, 2011",0001713353,"So, you think you have problems? Things could ..."
3,5.0,"11 24, 2008",0001713353,My mom bought this for my oldest son. He and ...
4,3.0,"09 18, 2006",0001713353,41 years later:\n\nThe cheese is government ch...
...,...,...,...,...
115439,5.0,"11 27, 1999",0030554179,"""All animals are equal, but some are more equa..."
115440,5.0,"09 9, 1999",0030554179,It's a story about animals who take over a far...
115441,5.0,"07 27, 1999",0030554179,We should not be content to reduce this book t...
115442,5.0,"06 5, 1999",0030554179,George Orwell is the pseudonym of Eric A. Blai...


In [6]:
# Unique set of books to crawl from Goodreads
books = set(amazon.asin)
print(len(books))

3007


In [9]:
# Create empty dataframe to store Goodreads reviews & ratings
column_names = ['overall', 'reviewTime', 'asin', 'reviewText']
df = pd.DataFrame(columns = column_names)
print(df)

Empty DataFrame
Columns: [overall, reviewTime, asin, reviewText]
Index: []


In [10]:
# Goodreads API Developer key
key = 'mTBsGSOvfBaN0OF25U5g'
secret = 'AkIOvUnGYksPPicdwWfQVStbr61O0YzzrVu1ZUmpGA'

In [11]:
# Log in Goodreads
driver = webdriver.Chrome()
driver.get ('https://www.goodreads.com/user/sign_in?source=home')
driver.find_element_by_name('user[email]').send_keys('@uci.edu') ## ENTER EMAIL
driver.find_element_by_name('user[password]').send_keys('') ## ENTER PASSWORD
driver.find_element_by_name('next').click()                            

In [12]:
# Set of books not found on Goodreads:
notfound = set()

In [13]:
# Get most popular reviews for each book (10 if there are more than 10 reviews, or all reviews if less)
print(books)
for book in books:
    # Try to look up the book in Goodreads. If a book isn't found it will not be added to the dataset -> DATA CLEANING LATER
    try:
        res = requests.get('https://www.goodreads.com/book/show.json', params={'key': key, 'id': book, 'text_only': True})
        data = res.json()
        # Parse HTML JSON to find link to reviews
        soup = BeautifulSoup(data['reviews_widget'], 'html.parser')
        url = soup.find('iframe')['src']
        # Open link to reviews using Selenium
        driver.get(url)
        # Expand each review to save full text & metadata
        links = driver.find_elements_by_link_text('...more')
        # Count number of reviews to make sure there are <= 10 reviews for each book
        count = 0
        for link in links:
            if count == 10:
                break
            ActionChains(driver).move_to_element(link).perform()
            link.click() # If a book doesn't have reviews, it will not be added to the dataset -> DATA CLEANING LATER
            count += 1
        for i in range(count):
            driver.switch_to.window(driver.window_handles[1])
            s = BeautifulSoup(driver.page_source, 'html.parser')
            # Use BeautifulSoup to save rating
            rating = s.find('meta', itemprop='ratingValue')
            # Use BeautifulSoup to save reviewTime
            time = s.find('span', itemprop='datePublished')
            # Use BeautifulSoup to save text review
            review = s.find('div', itemprop='reviewBody')
            # Add review to dataframe
            df = df.append({'overall':rating.text, 'reviewTime':time.text.strip(), 'asin':book, 'reviewText':review.text.strip()}, ignore_index=True)
            driver.close()
    except:
        notfound.add(book)

{'0007364199', '000274080X', '003010274X', '0007337914', '0004707532'}
TRYING BOOK: 0007364199
COUNT: 0
TRYING BOOK: 000274080X
BOOK NOT FOUND: 000274080X
TRYING BOOK: 003010274X
COUNT: 0
TRYING BOOK: 0007337914
COUNT: 5
CURRENT URL: https://www.goodreads.com/review/show/768883795?utm_campaign=reviews&utm_medium=widget&utm_source=%28direct%29
CURRENT URL: https://www.goodreads.com/review/show/702251812?utm_campaign=reviews&utm_medium=widget&utm_source=%28direct%29
CURRENT URL: https://www.goodreads.com/review/show/1629787670?utm_campaign=reviews&utm_medium=widget&utm_source=%28direct%29
CURRENT URL: https://www.goodreads.com/review/show/2166992587?utm_campaign=reviews&utm_medium=widget&utm_source=%28direct%29
CURRENT URL: https://www.goodreads.com/review/show/59263833?utm_campaign=reviews&utm_medium=widget&utm_source=%28direct%29
TRYING BOOK: 0004707532
BOOK NOT FOUND: 0004707532


In [14]:
df

,overall,reviewTime,asin,reviewText
0,,"Nov 19, 2013",2956,This book is filled with lots of examples. Som...
1,,"Aug 23, 2013",2956,What can I say? This book secured me an A+ on ...
2,,"May 05, 2016",2956,This book is so helpful for beginners in resea...
3,,"Oct 28, 2017",2956,Very instructive. Very boring.
4,,"Jun 11, 2009",2956,A necessary evil this summer.


In [15]:
print(notfound)

{'000274080X', '0004707532'}
